## Importing Libraries

In [1]:
import os
import argparse
import logging
import yaml
from string import Template
import json
import random


import kfp
from kfp import dsl
from kfp.components import func_to_container_op

## Define Kubeflow Pipeline

In [2]:
@func_to_container_op
def show_results(accuracy :float) -> None:
    print(f"Accuracy of neural network: {accuracy}")

@dsl.pipeline(name='demo_pipeline', description='Fits a simple neural network')
def first_pipeline():

    download = kfp.components.load_component_from_file('download_data/download_data.yaml')
    preprocess_data = kfp.components.load_component_from_file('preprocess_data/preprocess_data.yaml')
    train_model = kfp.components.load_component_from_file('train_model/train_model.yaml')

    download_task = download()
    preprocess_data_task = preprocess_data(download_task.output)
    train_model_task = train_model(preprocess_data_task.output)
    
    show_results(train_model_task.outputs['Accuracy'])
    
if __name__ == '__main__':
    file_name = 'pipeline.yaml'
    kfp.compiler.Compiler().compile(first_pipeline, file_name)

## Upload pipeline

In [3]:
import kfp
# When not specified, host defaults to env var KF_PIPELINES_ENDPOINT.
# This is now equivalent to `client = kfp.Client(host='http://localhost:3000')`
client = kfp.Client(host='<my-host-url>')
print(client.list_experiments())

/home/joaquin/.local/lib/python3.8/site-packages/kfp/_client.py:223: UserWarning: The host <my-host-url> does not contain the "http" or "https" protocol. Defaults to "https".
  warnings.warn(


MaxRetryError: HTTPSConnectionPool(host='<my-host-url>', port=443): Max retries exceeded with url: /apis/v1beta1/healthz (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fc34b7d8730>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [12]:
import kfp
from kfp.components import create_component_from_func
import kfp.dsl as dsl

def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    return a + b


# create a component using the add function
add_op = create_component_from_func(add)

# compose your pipeline using the dsl.pipeline decorator
@dsl.pipeline(
    name='Addition pipeline',
    description='An example pipeline that performs addition calculations.')
def add_pipeline(
    a='1',
    b='7',
):
    first_add_task = add_op(a, 4)
    second_add_task = add_op(first_add_task.output, b)

# instantiate a client and submit your pipeline with arguments
client = kfp.Client(host='<my-host-url>')
client.create_run_from_pipeline_func(
    add_pipeline, arguments={
        'a': '7',
        'b': '8'
    })

/home/joaquin/.local/lib/python3.8/site-packages/kfp/_client.py:223: UserWarning: The host <my-host-url> does not contain the "http" or "https" protocol. Defaults to "https".
  warnings.warn(


MaxRetryError: HTTPSConnectionPool(host='<my-host-url>', port=443): Max retries exceeded with url: /apis/v1beta1/healthz (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f37aa49cd90>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [7]:
import kfp
client=kfp.Client()
#filepath = '/home/jovyan/AI-platform/kubeflow/example/' + file_name
filepath = os.getcwd() + file_name
name = 'kubeflow_demo_' + str(random.randint(30000,90000))
print("Uploaded pipeline:", name)
pipeline = client.pipeline_uploads.upload_pipeline(filepath, name=name)

782.06s - Error inserting pydevd breaks.
Traceback (most recent call last):
  File "/home/joaquin/.local/lib/python3.8/site-packages/debugpy/_vendored/pydevd/_pydevd_frame_eval/pydevd_modify_bytecode.py", line 328, in insert_pydevd_breaks
    for new_instruction in get_instructions_to_add(
  File "/home/joaquin/.local/lib/python3.8/site-packages/debugpy/_vendored/pydevd/_pydevd_frame_eval/pydevd_modify_bytecode.py", line 102, in get_instructions_to_add
    Instr("LOAD_CONST", _pydev_stop_at_break, lineno=stop_at_line - 1),
  File "/home/joaquin/.local/lib/python3.8/site-packages/debugpy/_vendored/pydevd/_pydevd_frame_eval/vendored/bytecode/instr.py", line 171, in __init__
    self._set(name, arg, lineno)
  File "/home/joaquin/.local/lib/python3.8/site-packages/debugpy/_vendored/pydevd/_pydevd_frame_eval/vendored/bytecode/instr.py", line 239, in _set
    _check_lineno(lineno)
  File "/home/joaquin/.local/lib/python3.8/site-packages/debugpy/_vendored/pydevd/_pydevd_frame_eval/vendored/by

MaxRetryError: HTTPSConnectionPool(host='kubernetes.docker.internal', port=6443): Max retries exceeded with url: /api/v1/namespaces/kubeflow/services/ml-pipeline:http/proxy//apis/v1beta1/healthz (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f37aa5a1b50>: Failed to establish a new connection: [Errno 111] Connection refused'))